### 聊天机器人

In [1]:
import requests
import itchat

KEY = ''

def get_response(msg):
    # 这里我们就像在“3. 实现最简单的与图灵机器人的交互”中做的一样
    # 构造了要发送给服务器的数据
    apiUrl = 'http://www.tuling123.com/openapi/api'
    data = {
        'key'    : KEY,
        'info'   : msg,
        'userid' : 'wechat-robot',
    }
    try:
        r = requests.post(apiUrl, data=data).json()
        # 字典的get方法在字典没有'text'值的时候会返回None而不会抛出异常
        return r.get('text')
    # 为了防止服务器没有正常响应导致程序异常退出，这里用try-except捕获了异常
    # 如果服务器没能正常交互（返回非json或无法连接），那么就会进入下面的return
    except:
        # 将会返回一个None
        return

# 这里是我们在“1. 实现微信消息的获取”中已经用到过的同样的注册方法
@itchat.msg_register(itchat.content.TEXT)
def tuling_reply(msg):
    # 为了保证在图灵Key出现问题的时候仍旧可以回复，这里设置一个默认回复
    defaultReply = 'I received: ' + msg['Text']
    # 如果图灵Key出现问题，那么reply将会是None
    reply = get_response(msg['Text'])
    # a or b的意思是，如果a有内容，那么返回a，否则返回b
    # 有内容一般就是指非空或者非None，你可以用`if a: print('True')`来测试
    return reply or defaultReply

# 为了让实验过程更加方便（修改程序不用多次扫码），我们使用热启动
itchat.auto_login(hotReload=True)
itchat.run()

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as 罗攀
Start auto replying.
LOG OUT!


### 性别预测

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [3]:
all_data = pd.read_csv('friend.csv',encoding='utf-8')
all_data.head()

,NickName,Sex,Province,City,Signature
0,☀️,2,湖北,武汉,NaN
1,郭成,1,湖南,常德,NaN
2,彭彬,1,云南,昆明,啦啦啦
3,科莫呗比,1,湖南,常德,NaN
4,Q,2,湖南,常德,如果再见不能红着脸 是否还能红着眼


In [5]:
data = all_data[(all_data['Sex'] == 1) | (all_data['Sex'] == 2)]
data.head(10)

,NickName,Sex,Province,City,Signature
0,☀️,2,湖北,武汉,NaN
1,郭成,1,湖南,常德,NaN
2,彭彬,1,云南,昆明,啦啦啦
3,科莫呗比,1,湖南,常德,NaN
4,Q,2,湖南,常德,如果再见不能红着脸 是否还能红着眼
7,一袭水袖舞倾城,2,NaN,NaN,谁的青春不迷茫
8,夏夜星空,2,湖南,常德,NaN
9,此号停用,2,湖南,常德,做自己让以后的自己感谢的事！
10,夏,1,Abu Dhabi,NaN,NaN
11,盐析菜,1,NaN,NaN,NaN


In [6]:
all_data.shape

(469, 5)

In [7]:
data.shape

(448, 5)

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(data['NickName'], data['Sex'], test_size=0.2, random_state=22)

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_cov = count_vect.fit_transform(X_train)

In [10]:
clf = MultinomialNB(alpha=0.0001)
clf.fit(X_train_cov, Y_train)

MultinomialNB(alpha=0.0001, class_prior=None, fit_prior=True)

In [11]:
clf.score(X_train_cov, Y_train)

0.96368715083798884

In [12]:
X_test_cov = count_vect.transform(X_test)
clf.score(X_test_cov, Y_test)

0.57777777777777772

In [13]:
test = ['陈傻逼','罗罗攀']
X = count_vect.transform(test)
clf.predict(X)

array([1, 1], dtype=int64)